# SNN (Nengo/NengoDL) su NSL-KDD — Colab

Notebook end-to-end: clona la repo, installa requisiti, scarica NSL-KDD, preprocessing, rete Nengo con neuroni LIF, training con NengoDL, valutazione e salvataggio parametri.

- Repo: https://github.com/devedale/snn-ids
- Framework: Nengo + NengoDL (TensorFlow backend)
- Codifica: rate coding (ripetizione su T step)
- Task: binaria (anomalo vs normale)

**Nota sui conflitti pip**: I warning di dependency conflicts sono normali in Colab. NengoDL richiede TensorFlow 2.13.1 + numpy 1.24.3 che confliggono con alcuni pacchetti preinstallati, ma il notebook funziona comunque.


In [ ]:
# Cleanup ambiente + install compatibili (NengoDL) e riavvio runtime
!rm -rf /content/snn-ids && git clone https://github.com/devedale/snn-ids.git /content/snn-ids -q
%cd /content/snn-ids

# Rimuove pacchetti preinstallati che confliggono con TF/Keras 2.x
!pip -q uninstall -y jax jaxlib tensorflow-text tensorflow-decision-forests tensorflow-hub \
  orbax-checkpoint chex optax flax dopamine-rl tensorstore || true
!pip -q uninstall -y tensorflow tensorflow-cpu tensorflow-gpu keras-hub opencv-python opencv-contrib-python opencv-python-headless || true
!pip -q uninstall -y keras keras-nightly tf-keras keras-preprocessing keras-vis || true
!pip -q uninstall -y albucore albumentations ydf grpcio-status typeguard torch torchvision torchaudio fastai sentence-transformers peft timm accelerate || true

# Aggiorna toolchain e installa requisiti allineati a NengoDL (TF/Keras 2.13.x)
!pip -q install -U pip setuptools wheel jedi==0.18.2
!pip -q install -r requirements_colab_nengo.txt

print("✅ Installazione completata. I warning di dependency conflicts sono normali e non bloccano l'esecuzione.")
print("ℹ️  Adesso riavvio il runtime per applicare le modifiche...")

import os
os.kill(os.getpid(), 9)


In [ ]:
# Verifica versioni (dopo riavvio runtime)
!rm -rf /content/snn-ids && git clone https://github.com/devedale/snn-ids.git /content/snn-ids -q
%cd /content/snn-ids

import tensorflow as tf, keras
print("✅ TensorFlow:", tf.__version__, "| Keras:", keras.__version__)

# Test import NengoDL per verificare che keras.engine sia accessibile
try:
    import nengo, nengo_dl
    print("✅ Nengo:", nengo.__version__, "| NengoDL:", nengo_dl.__version__)
    print("✅ Importazioni completate con successo!")
except Exception as e:
    print("❌ Errore import NengoDL:", e)


In [ ]:
# Download NSL-KDD (train/test) e conversione CSV→dataset SNN via tool repo
from pathlib import Path
import subprocess

base = Path('/content/data')
base.mkdir(parents=True, exist_ok=True)

# Download dei file NSL-KDD
urls = {
    'KDDTrain+': 'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt',
    'KDDTest+':  'https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTest+.txt',
}

print("📥 Download NSL-KDD dataset...")
for name, url in urls.items():
    !wget -q -O /content/data/{name}.txt "{url}"

print("🔄 Conversione in formato SNN...")
# Usa il tool della repo per convertire i CSV in formato SNN
!python -m tools.csv_to_snn_dataset --input /content/data/KDDTrain+.txt --output /content/data/nslkdd_snn_train.csv --label-col label
!python -m tools.csv_to_snn_dataset --input /content/data/KDDTest+.txt --output /content/data/nslkdd_snn_test.csv --label-col label

print("📂 File creati:")
!ls -lh /content/data | head -n 10


In [ ]:
# Carica dataset SNN + label reali dai raw
import numpy as np, pandas as pd

print("📊 Caricamento dataset SNN preprocessati...")
train_df = pd.read_csv('/content/data/nslkdd_snn_train.csv')
test_df  = pd.read_csv('/content/data/nslkdd_snn_test.csv')

# Label dai file raw originali per maggiore accuratezza
cols = [
  "duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent",
  "hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root",
  "num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login",
  "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
  "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate",
  "dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate",
  "dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","label","difficulty"
]

print("🏷️  Estrazione label originali...")
raw_train = pd.read_csv('/content/data/KDDTrain+.txt', names=cols)
raw_test  = pd.read_csv('/content/data/KDDTest+.txt',  names=cols)
y_train = (raw_train['label'].astype(str).str.lower() != 'normal').astype(int).values
y_test  = (raw_test['label'].astype(str).str.lower()  != 'normal').astype(int).values

# Estrai features dal dataset SNN (escludendo timestamp)
feat_cols = [c for c in train_df.columns if c != 'timestamp']
X_train = train_df[feat_cols].values.astype(np.float32)
X_test  = test_df[feat_cols].values.astype(np.float32)

input_size, output_size = X_train.shape[1], 2
print(f"📈 Shape dataset: Train {X_train.shape}, Test {X_test.shape}")
print(f"🧠 Configurazione rete: {input_size} input → {output_size} output (binary classification)")
print(f"🎯 Distribuzione label: Train {np.bincount(y_train)}, Test {np.bincount(y_test)}")


In [ ]:
# Costruzione rete Nengo e training con NengoDL
import nengo, nengo_dl, tensorflow as tf
import numpy as np

print("🔧 Costruzione rete Nengo...")

# Parametri della rete
timesteps = 10
n_hidden = 256

# Costruzione della rete SNN con Nengo
with nengo.Network() as net:
    # Input layer (features)
    inp = nengo.Node(np.zeros(input_size))
    
    # Hidden layer con neuroni LIF
    ens1 = nengo.Ensemble(n_neurons=n_hidden, dimensions=input_size, neuron_type=nengo.LIF())
    nengo.Connection(inp, ens1)
    
    # Output layer (classificazione binaria)
    ens2 = nengo.Ensemble(n_neurons=output_size, dimensions=output_size, neuron_type=nengo.LIF())
    nengo.Connection(ens1, ens2)
    
    # Probe per raccogliere l'output
    p_out = nengo.Probe(ens2, synapse=0.1)

print("🔄 Encoding per rate coding (ripetizione features su timesteps)...")
# Rate coding: ripeti le features per T timesteps
X_train_enc = np.repeat(X_train[:, None, :], timesteps, axis=1)
X_test_enc  = np.repeat(X_test[:,  None, :], timesteps, axis=1)

# One-hot encoding delle label per MSE loss
y_train_oh = tf.keras.utils.to_categorical(y_train, num_classes=output_size)
y_test_oh  = tf.keras.utils.to_categorical(y_test,  num_classes=output_size)
y_train_enc = np.repeat(y_train_oh[:, None, :], timesteps, axis=1)
y_test_enc  = np.repeat(y_test_oh[:,  None, :], timesteps, axis=1)

print(f"📊 Shape encoded: X_train {X_train_enc.shape}, y_train {y_train_enc.shape}")

print("🚀 Avvio training con NengoDL...")
minibatch = 64  # Ridotto per evitare OOM su Colab
epochs = 3      # Ridotto per demo rapida

with nengo_dl.Simulator(net, minibatch_size=minibatch) as sim:
    # Configurazione training
    inputs = {inp: X_train_enc}
    targets = {p_out: y_train_enc}
    
    sim.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss=tf.keras.losses.MSE)
    
    # Training con validation split
    sim.fit(inputs, targets, epochs=epochs, validation_split=0.2, verbose=2)
    
    print("🧪 Valutazione su test set...")
    # Inferenza sui dati di test
    test_dict = {inp: X_test_enc}
    sim.run_steps(X_test_enc.shape[1], data=test_dict)
    out = sim.data[p_out]

# Post-processing predizioni
pred = out.mean(axis=1)  # Media su timesteps
y_pred = pred.argmax(axis=1)
acc = (y_pred == y_test).mean()

print(f"✅ Test accuracy: {acc:.4f}")
print(f"🎯 Predizioni: Normal {np.sum(y_pred == 0)}, Anomaly {np.sum(y_pred == 1)}")


In [ ]:
# Salvataggio artefatti del modello
import os, pickle
from sklearn.metrics import classification_report, confusion_matrix

print("📊 Report di classificazione dettagliato:")
print(classification_report(y_test, y_pred, target_names=['Normal', 'Anomaly']))

print("🔢 Confusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(f"   Predicted:  Normal  Anomaly")
print(f"Actual Normal:   {cm[0,0]:4d}    {cm[0,1]:4d}")
print(f"     Anomaly:   {cm[1,0]:4d}    {cm[1,1]:4d}")

# Salva la rete e le metriche
print("💾 Salvataggio artefatti...")
os.makedirs('/content/out', exist_ok=True)

# Salva la rete Nengo
with open('/content/out/nengo_nslkdd_network.pkl', 'wb') as f:
    pickle.dump(net, f)

# Salva i risultati e metriche
results = {
    'accuracy': acc,
    'predictions': y_pred,
    'true_labels': y_test,
    'confusion_matrix': cm,
    'network_config': {
        'input_size': input_size,
        'hidden_neurons': n_hidden,
        'output_size': output_size,
        'timesteps': timesteps,
        'epochs': epochs,
        'batch_size': minibatch
    }
}

with open('/content/out/nengo_nslkdd_results.pkl', 'wb') as f:
    pickle.dump(results, f)

print("✅ File salvati in /content/out/:")
print("   - nengo_nslkdd_network.pkl (rete Nengo)")
print("   - nengo_nslkdd_results.pkl (risultati e metriche)")
!ls -lh /content/out/
